In [186]:
import pandas as pd
import numpy as np

In [187]:
data = pd.read_json("data/response.json")

In [188]:
data.head(1)

,combo_products,name,category,sub_category,cuisine,brand,vendor,desc,short_desc,file,...,seo_title,seo_description,seo_keywords,top,taste,slug_history,_id,point_exp_date,slug,__v
0,NaN,"Pyaz Kachori (6 pieces) - Radha Kachori, Jaipur","{'name': 'COOKED FOOD', '_id': '6050bb1256cb9d...","{'name': 'Cooked Vegetarian Food Items', '_id'...","{'name': 'Rajasthani', '_id': '604b4a98dbd6250...","{'name': 'Radhe Kachori - Jaipur', '_id': '617...","{'full_name': 'RADHE KACHORI - JAIPUR', '_id':...",Kanda Kachauri is a kind of Rajasthani Kachori...,Kanda Kachauri is a kind of Rajasthani Kachori...,"[{'fieldname': 'upload', 'originalname': 'Pyaz...",...,Pyajj Kachori Radhe Kachori Jaipur | tastes2plate,Order online Intercity delivery of Pyaz Kachor...,"Pyaz Kachori Radhe Kachori Jaipur, Pyaz Kachor...",0,1,[pyaz-kachori-6-pieces-radha-kachori-jaipur],652913a6e6e20a661fa66975,2023-10-13T00:00:00.000Z,pyaz-kachori-6-pieces-radha-kachori-jaipur,0


In [189]:
## Extract valueable info from this data for ex category has 2 fields name and id so we extract name

In [190]:
data['category'].value_counts()
## data is imbalanced so we can improve our model here also

{'name': 'COOKED FOOD', '_id': '6050bb1256cb9d00088521bc'}    2453
{'name': 'SWEET', '_id': '6050bf608f37b70008b0ed38'}           196
Name: category, dtype: int64

In [191]:
def extractInfo(info, key):
    return info[key]

In [192]:
data['category'] =  data['category'].apply(extractInfo, key="name")
data['sub_category'] = data['sub_category'].apply(extractInfo, key="name")
data['cuisine'] = data['cuisine'].apply(extractInfo, key='name')
data['brand'] = data['brand'].apply(extractInfo, key='name')
data['vendor'] = data['vendor'].apply(extractInfo, key='full_name')
data['city'] = data['city'].apply(extractInfo, key='name')
data['file']= data['file'].apply(lambda x:x[0]['location'])


In [193]:
data.head(1)

,combo_products,name,category,sub_category,cuisine,brand,vendor,desc,short_desc,file,...,seo_title,seo_description,seo_keywords,top,taste,slug_history,_id,point_exp_date,slug,__v
0,NaN,"Pyaz Kachori (6 pieces) - Radha Kachori, Jaipur",COOKED FOOD,Cooked Vegetarian Food Items,Rajasthani,Radhe Kachori - Jaipur,RADHE KACHORI - JAIPUR,Kanda Kachauri is a kind of Rajasthani Kachori...,Kanda Kachauri is a kind of Rajasthani Kachori...,https://t2p.fra1.digitaloceanspaces.com/169719...,...,Pyajj Kachori Radhe Kachori Jaipur | tastes2plate,Order online Intercity delivery of Pyaz Kachor...,"Pyaz Kachori Radhe Kachori Jaipur, Pyaz Kachor...",0,1,[pyaz-kachori-6-pieces-radha-kachori-jaipur],652913a6e6e20a661fa66975,2023-10-13T00:00:00.000Z,pyaz-kachori-6-pieces-radha-kachori-jaipur,0


In [194]:
data.columns

Index(['combo_products', 'name', 'category', 'sub_category', 'cuisine',
       'brand', 'vendor', 'desc', 'short_desc', 'file', 'price',
       'selling_price', 'discounted_price', 'purchase_price', 'batchno',
       'city', 'commission', 'packaging_charge', 'tax_status', 'cgst', 'sgst',
       'igst', 'sku', 'stock_qty', 'backorders', 'threshold', 'manage_stock',
       'weight', 'length', 'width', 'height', 'attribute', 'tags',
       'start_date', 'end_date', 'created_date', 'update_date', 'active',
       'deal', 'featured', 'deleted', 'express', 'added_by', 'point',
       'stock_product', 'consumable', 'seo_title', 'seo_description',
       'seo_keywords', 'top', 'taste', 'slug_history', '_id', 'point_exp_date',
       'slug', '__v'],
      dtype='object')

In [28]:
## Remove Unwanted columns

In [195]:
data = data[['name', 'category', 'sub_category', 'cuisine', 'brand', 'desc', 'short_desc', '_id', 'city', 'tags', 'file']]

In [196]:
data.head(1)

,name,category,sub_category,cuisine,brand,desc,short_desc,_id,city,tags,file
0,"Pyaz Kachori (6 pieces) - Radha Kachori, Jaipur",COOKED FOOD,Cooked Vegetarian Food Items,Rajasthani,Radhe Kachori - Jaipur,Kanda Kachauri is a kind of Rajasthani Kachori...,Kanda Kachauri is a kind of Rajasthani Kachori...,652913a6e6e20a661fa66975,JAIPUR,"Pyaz Kachori Jaipur, Pyaz Kachori Radhe Kachor...",https://t2p.fra1.digitaloceanspaces.com/169719...


In [197]:
# in some columns there are html text so we have to parse it and make a simple text
from bs4 import BeautifulSoup

In [198]:
def cleanText(text):
    soup = BeautifulSoup(text, "html.parser")
    parsed_text = soup.get_text()
    # Replace newline characters with spaces
    parsed_text = parsed_text.replace('\r\n', ' ')

    # Replace non-breaking spaces with regular spaces
    parsed_text = parsed_text.replace('\xa0', ' ')
    
    return parsed_text

In [199]:
data['desc'] = data['desc'].apply(cleanText)
data['category'] = data['category'].apply(cleanText)
data['sub_category'] = data['sub_category'].apply(cleanText)
data['cuisine'] = data['cuisine'].apply(cleanText)
data['brand'] = data['brand'].apply(cleanText)
data['desc'] = data['desc'].apply(cleanText)
data['short_desc'] = data['short_desc'].apply(cleanText)
data['city'] = data['city'].apply(cleanText)
data['tags'] = data['tags'].apply(cleanText)

C:\Users\akkik\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [ ]:
## now we make a column with 'tags' name and add all the string column and remove them

In [200]:
data['tags'] = data['name']+" "+data['category']+" "+data['cuisine']+" "+data['brand']+" "+data['desc']+" "+data['short_desc']+" "+data['city']+" "+data['tags']

In [201]:
data = data[['_id', 'name', 'tags', 'file']]

In [202]:
data.head()

,_id,name,tags,file
0,652913a6e6e20a661fa66975,"Pyaz Kachori (6 pieces) - Radha Kachori, Jaipur","Pyaz Kachori (6 pieces) - Radha Kachori, Jaipu...",https://t2p.fra1.digitaloceanspaces.com/169719...
1,65291eede6e20a661fa692e3,"Pyaz Kachori (4 pieces) - Radhe Kachori, Jaipur","Pyaz Kachori (4 pieces) - Radhe Kachori, Jaipu...",https://t2p.fra1.digitaloceanspaces.com/169719...
2,646231c3c7a5bd83862a15d5,"Bheja Fry - Chicha's, Lakdikapul, Hyderabad","Bheja Fry - Chicha's, Lakdikapul, Hyderabad CO...",https://t2p.fra1.digitaloceanspaces.com/168415...
3,6462356cc7a5bd83862a17a2,"Butter Chicken (Boneless) - Chicha's, Lakdikap...","Butter Chicken (Boneless) - Chicha's, Lakdikap...",https://t2p.fra1.digitaloceanspaces.com/168415...
4,64623699c7a5bd83862a18e9,"Chicken 65 - Chicha's, Lakdikapul, Hyderabad","Chicken 65 - Chicha's, Lakdikapul, Hyderabad C...",https://t2p.fra1.digitaloceanspaces.com/168415...


In [111]:
# convert all to lower case

In [203]:
data['tags'] = data['tags'].apply(lambda x:x.lower())

C:\Users\akkik\AppData\Local\Temp\ipykernel_10280\3172751869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(lambda x:x.lower())


In [115]:
# steming of words

In [204]:
import nltk

In [205]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [206]:
def stem(text):
    L = []
    
    for i in text.split():
        L.append(ps.stem(i))
    
    return " ".join(L)

In [207]:
data['tags'] = data['tags'].apply(stem)

C:\Users\akkik\AppData\Local\Temp\ipykernel_10280\2997483169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(stem)


In [208]:
# now convert this string to vector and remove stop words

In [209]:
from sklearn.feature_extraction.text import CountVectorizer

In [210]:
cv = CountVectorizer(max_features = 3000, stop_words='english')

In [211]:
vectors = cv.fit_transform(data['tags']).toarray()

In [212]:
vectors.shape

(2649, 3000)

In [213]:
# now we got similarity between movies

In [214]:
from sklearn.metrics.pairwise import cosine_similarity

In [215]:
similarity = cosine_similarity(vectors)

In [216]:
similarity.shape

(2649, 2649)

In [217]:
def recommend(food):
    print("food is : " + food)
    food_index = data[data['name'] == food].index[0]
    distances = similarity[food_index]
    food_list = sorted(list(enumerate(distances)), reverse = True, key=lambda x:x[1])[1:20]
    
    for i in food_list:
        print(data.iloc[i[0]]['name'])

In [218]:
recommend(data['name'][2300])

food is : Chicken Seekh Kebab (1 Pc)- Hotel Muhammadi Palace, Jaipur
Chicken Seekh Kebab (2 Pcs)- Hotel Muhammadi Palace, Jaipur
Chicken Seekh Kebab (4Pcs) - Mughal Zaika, Lucknow
Chicken Seekh Kabab- Dastarkhwan (Lucknow)
Chicken Seekh Kebab (Half) - Open Air Restaurant, Lucknow
Chicken Tangdi (4 Pcs) - Hotel Muhammadi Palace, Jaipur
Chicken Tangdi (2 Pcs) - Hotel Muhammadi Palace, Jaipur
Chicken Bone Less(Full Plate) - Hotel Muhammadi Palace, Jaipur
Chicken Bone Less(Half Plate) - Hotel Muhammadi Palace, Jaipur
Chicken Seekh Kebab (Full) - Open Air Restaurant, Lucknow
Chicken Stew (2 Pcs)- Hotel Muhammadi Palace, Jaipur
Chicken Stew (1 Pc)- Hotel Muhammadi Palace, Jaipur
Chicken Tikka (4 Pcs) - Hotel Muhammadi Palace, Jaipur
Chicken Sheek Kabab (8 Pcs) - Mandar, Hyderabad
Chicken Fry (2 Pcs) - Hotel Muhammadi Palace, Jaipur
Butter Chicken (2 Pcs) - Muhammadi Palace, Jaipur
Butter Chicken (4 Pcs) - Muhammadi Palace, Jaipur
Chicken Tikka (8 Pcs) - Hotel Muhammadi Palace, Jaipur
Chicken

In [170]:
#now make pickle file

In [219]:
import pickle

In [220]:
pickle.dump(data.to_dict(), open('food_dict.pkl', 'wb'))

In [221]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))